> I turned around to see this cute guy holding an item I had bought. He said, ‘I got the same thing!’ We laughed about it and wound up swapping items because I wanted the color he got.
> I asked him to get some food with me and we spent the rest of the day together.

In [229]:
import polars as pl

customers = pl.read_csv("harder-data/noahs-customers.csv",try_parse_dates=True)
orders_items = pl.read_csv("harder-data/noahs-orders_items.csv",try_parse_dates=True)
orders = pl.read_csv("harder-data/noahs-orders.csv",try_parse_dates=True)
products = pl.read_csv("harder-data/noahs-products.csv",try_parse_dates=True)

Step 1: Find all the items with colors in their names. 
After inspecting the products table, it seems that colors are written in **lower-case** at the end of the product description in parentheses.
I'll separate the product color into its own column.

In [237]:
# Products with colors in the name always end in e.g. `... (blue)`, so use a regex that pulls that color name out
color_name_filter = r"(.+) \(([[:lower:]]+)\)"
colorful_products = products.filter(pl.col("desc").str.contains(color_name_filter))
products_with_colors_extracted = colorful_products.select(
    pl.col("sku"),
    pl.col("desc").str.extract(color_name_filter, group_index=1),
    pl.col("desc").str.extract(color_name_filter, group_index=2).alias("color")
)
print(products_with_colors_extracted)

shape: (423, 3)
┌─────────┬──────────────────────┬────────┐
│ sku     ┆ desc                 ┆ color  │
│ ---     ┆ ---                  ┆ ---    │
│ str     ┆ str                  ┆ str    │
╞═════════╪══════════════════════╪════════╡
│ HOM0017 ┆ Vintage Widget       ┆ azure  │
│ COL0030 ┆ Noah's Action Figure ┆ azure  │
│ HOM0042 ┆ Vintage Toaster      ┆ red    │
│ COL0061 ┆ Noah's Jewelry       ┆ yellow │
│ HOM0079 ┆ Vintage Radio        ┆ purple │
│ …       ┆ …                    ┆ …      │
│ HOM9819 ┆ Mini Machine         ┆ puce   │
│ HOM9840 ┆ Vintage Blender      ┆ green  │
│ HOM9899 ┆ Mini Machine         ┆ green  │
│ HOM9920 ┆ Vintage Bicycle      ┆ orange │
│ HOM9997 ┆ Handmade Machine     ┆ mauve  │
└─────────┴──────────────────────┴────────┘


Ok now I'll find all orders with items that contain "colorful products".

In [231]:
orders_with_colorful_products = orders_items.join(products_with_colors_extracted, on="sku", how="inner")
print(orders_with_colorful_products)

shape: (192_789, 6)
┌─────────┬─────────┬─────┬────────────┬───────────────────┬─────────┐
│ orderid ┆ sku     ┆ qty ┆ unit_price ┆ desc              ┆ color   │
│ ---     ┆ ---     ┆ --- ┆ ---        ┆ ---               ┆ ---     │
│ i64     ┆ str     ┆ i64 ┆ f64        ┆ str               ┆ str     │
╞═════════╪═════════╪═════╪════════════╪═══════════════════╪═════════╡
│ 1001    ┆ COL5420 ┆ 1   ┆ 12.27      ┆ Noah's Jersey     ┆ mauve   │
│ 1003    ┆ HOM3460 ┆ 1   ┆ 5.83       ┆ Mini Widget       ┆ azure   │
│ 1005    ┆ HOM4176 ┆ 1   ┆ 70.01      ┆ Toy Bicycle       ┆ purple  │
│ 1005    ┆ HOM2950 ┆ 1   ┆ 11.23      ┆ Vintage Widget    ┆ purple  │
│ 1006    ┆ COL7392 ┆ 1   ┆ 5.17       ┆ Noah's Bobblehead ┆ red     │
│ …       ┆ …       ┆ …   ┆ …          ┆ …                 ┆ …       │
│ 252097  ┆ HOM9819 ┆ 1   ┆ 42.23      ┆ Mini Machine      ┆ puce    │
│ 252099  ┆ HOM5251 ┆ 1   ┆ 55.55      ┆ Toy Bicycle       ┆ azure   │
│ 252099  ┆ HOM8156 ┆ 1   ┆ 47.65      ┆ Handmade Radio  

And now I have to find consecutive orders (the orderids auto-increment by one) which have the same item but a different color...
This is pretty hard for me conceptually, because I don't know how to compare rows only if they're sequential.

Maybe I could do a group_by on "desc", collect the orderids into a list, and somehow find the lists that have orderid off by one with same "desc"??

In [232]:
orders_by_colorful_products = (
    orders_with_colorful_products.group_by("desc")
    .agg(pl.col("orderid").sort()) # Sort orders for pairwise filter later
    .sort("desc")
)
print(orders_by_colorful_products)

shape: (36, 2)
┌──────────────────┬────────────────────────┐
│ desc             ┆ orderid                │
│ ---              ┆ ---                    │
│ str              ┆ list[i64]              │
╞══════════════════╪════════════════════════╡
│ Handmade Machine ┆ [1175, 1201, … 251935] │
│ Handmade Radio   ┆ [1023, 1268, … 252099] │
│ Mini Bicycle     ┆ [1024, 1045, … 252089] │
│ Mini Blender     ┆ [1023, 1052, … 252093] │
│ Mini Crockpot    ┆ [1040, 1048, … 252086] │
│ …                ┆ …                      │
│ Vintage Mixer    ┆ [1084, 1149, … 252034] │
│ Vintage Radio    ┆ [1024, 1060, … 252090] │
│ Vintage Timer    ┆ [1100, 1203, … 252047] │
│ Vintage Toaster  ┆ [1070, 1099, … 252079] │
│ Vintage Widget   ┆ [1005, 1010, … 251978] │
└──────────────────┴────────────────────────┘


For each product, I must find all the order ids that are consecutive.
This is called going pairwise through the list, and luckily python itertools implements that for me.

In [233]:
from itertools import pairwise

def filter_consecutive_ids(series: pl.List(pl.Int64)) -> list[tuple[int, int]]:
    output = []
    for pair in pairwise(series):
        if pair[1] - pair[0] == 1:
            output.append(pair)
    return output

items_ordered_consecutively = orders_by_colorful_products.select(
    pl.col("desc"),
    pl.col("orderid").map_elements(
        filter_consecutive_ids, return_dtype=pl.List(pl.List(pl.Int64))
    ).alias("consecutive_orderid"),
).explode("consecutive_orderid")
print(items_ordered_consecutively)


shape: (4_071, 2)
┌──────────────────┬─────────────────────┐
│ desc             ┆ consecutive_orderid │
│ ---              ┆ ---                 │
│ str              ┆ list[i64]           │
╞══════════════════╪═════════════════════╡
│ Handmade Machine ┆ [6271, 6272]        │
│ Handmade Machine ┆ [8516, 8517]        │
│ Handmade Machine ┆ [8813, 8814]        │
│ Handmade Machine ┆ [16304, 16305]      │
│ Handmade Machine ┆ [17989, 17990]      │
│ …                ┆ …                   │
│ Vintage Widget   ┆ [245232, 245233]    │
│ Vintage Widget   ┆ [249292, 249293]    │
│ Vintage Widget   ┆ [250428, 250429]    │
│ Vintage Widget   ┆ [250864, 250865]    │
│ Vintage Widget   ┆ [251758, 251759]    │
└──────────────────┴─────────────────────┘


This is a table of all the consecutive orders where a specific item (ignoring its color) was bought.
I now need to filter for the rows where the color changed between each order in the list.

In [234]:
def get_orderids_to_item_color(series: pl.List(pl.Int64)) -> pl.List(pl.String):
    # orders_with_colorful_products is a global variable :)
    return orders_with_colorful_products.filter(pl.col("orderid").is_in(series)).select(
        "color"
    )

# - Add a 'color' column that has the color of each item from the corresponding orderid list
# - Filter for rows where exactly one item was bought in each order (length 2)
# - Filter for rows where the color list has two unique elements
items_ordered_consecutively_with_color = items_ordered_consecutively.select(
    pl.col("desc"),
    pl.col("consecutive_orderid"),
    pl.col("consecutive_orderid")
    .map_elements(get_orderids_to_item_color, return_dtype=pl.List(pl.String))
    .alias("color"),
).filter(pl.col("color").list.len() == 2).filter(pl.col("color").list.n_unique() == 2)
print(items_ordered_consecutively_with_color)

shape: (1_008, 3)
┌──────────────────┬─────────────────────┬───────────────────────┐
│ desc             ┆ consecutive_orderid ┆ color                 │
│ ---              ┆ ---                 ┆ ---                   │
│ str              ┆ list[i64]           ┆ list[str]             │
╞══════════════════╪═════════════════════╪═══════════════════════╡
│ Handmade Machine ┆ [8516, 8517]        ┆ ["puce", "white"]     │
│ Handmade Machine ┆ [32642, 32643]      ┆ ["orange", "magenta"] │
│ Handmade Machine ┆ [43746, 43747]      ┆ ["amber", "mauve"]    │
│ Handmade Machine ┆ [51362, 51363]      ┆ ["white", "magenta"]  │
│ Handmade Machine ┆ [75567, 75568]      ┆ ["white", "puce"]     │
│ …                ┆ …                   ┆ …                     │
│ Vintage Widget   ┆ [230343, 230344]    ┆ ["blue", "puce"]      │
│ Vintage Widget   ┆ [237791, 237792]    ┆ ["amber", "mauve"]    │
│ Vintage Widget   ┆ [240511, 240512]    ┆ ["purple", "amber"]   │
│ Vintage Widget   ┆ [250428, 250429]    ┆ [

I have the above table with 401 orders that are consecutive and have the same item in a different color.

I'll find the rows where either orderid belongs to Sherri Long, the woman from part 6.

In [238]:
#FIXME: Carry this over programatically
part6_customerid = 8884
part6_customer_orderids = (
    orders.filter(pl.col("customerid") == part6_customerid)
    .select("orderid")
    .to_series()
)

orders_from_part6_customer = items_ordered_consecutively_with_color.filter(
    pl.col("consecutive_orderid")
    .list.eval(pl.element().is_in(part6_customer_orderids))
    .list.any()
)
print(orders_from_part6_customer)

shape: (1, 3)
┌──────────────┬─────────────────────┬─────────────────────┐
│ desc         ┆ consecutive_orderid ┆ color               │
│ ---          ┆ ---                 ┆ ---                 │
│ str          ┆ list[i64]           ┆ list[str]           │
╞══════════════╪═════════════════════╪═════════════════════╡
│ Mini Machine ┆ [73256, 73257]      ┆ ["magenta", "blue"] │
└──────────────┴─────────────────────┴─────────────────────┘


I'll programatically get the order id + customer id of the customer that is not Sherri Long from the above list of orderids.

In [239]:
print(
    orders_from_part6_customer.explode("consecutive_orderid")
    .filter(pl.col("consecutive_orderid").is_in(part6_customer_orderids).not_())
    .join(orders, left_on="consecutive_orderid", right_on="orderid", how="inner")
    .join(customers, on="customerid", how="inner")
    .select("customerid", "name", "phone")
)


shape: (1, 3)
┌────────────┬──────────────┬──────────────┐
│ customerid ┆ name         ┆ phone        │
│ ---        ┆ ---          ┆ ---          │
│ i64        ┆ str          ┆ str          │
╞════════════╪══════════════╪══════════════╡
│ 9931       ┆ Jeremy Burch ┆ 516-544-4187 │
└────────────┴──────────────┴──────────────┘
